## BRANCH AND BOUND ALGORITMA

- Nama : Hafizh Hilman Asyhari
- NIM : 202331206
- Fakultas : Fakultas Telematika Energi
- Mata Kuliah : Analisis Algoritma
- Dosen : Bapak M Yoga Distra Sudirman, S.T., MTI
- Tahun : 2025
- Negara : Indonesia

In [ ]:
# Traveling Salesman Problem Dinamis dengan Visualisasi Upgrade
# --------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import heapq
from math import sqrt
import pandas as pd

In [2]:
# --- Input pengguna ---
nim = input("Masukkan NIM Anda: ").strip()
algoritma = input("Pilih algoritma (bb / greedy / semua): ").strip().lower()
jumlah_kota = int(input("Jumlah kota (default 5): ") or 5)

Masukkan NIM Anda:  202331206
Pilih algoritma (bb / greedy / semua):  bb
Jumlah kota (default 5):  5


In [3]:
# --- Parsing NIM berbasis blok ---
if len(nim) < 9:
    raise ValueError("NIM terlalu pendek. Butuh 9 digit untuk parsing blok.")
a = int(nim[0:2])
b = int(nim[2:4])
c = int(nim[4])
d = int(nim[5:7])
e = int(nim[7:9])

# --- Koordinat kota ---
if jumlah_kota == 5:
    coords = [(a,b), (b,c), (c,d), (d,e), (e,a)]
else:
    np.random.seed(int(nim[-3:]))
    coords = list(set((np.random.randint(0, 30), np.random.randint(0, 30)) for _ in range(jumlah_kota)))
    while len(coords) < jumlah_kota:
        coords.append((np.random.randint(0, 30), np.random.randint(0, 30)))
coords = coords[:jumlah_kota]

print("Koordinat kota:")
for i, coord in enumerate(coords):
    print(f"Kota {i}: {coord}")

# --- Matriks jarak ---
def euclidean(p1, p2):
    return round(sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2), 2)

n = len(coords)
dist_matrix = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        if i != j:
            dist_matrix[i][j] = euclidean(coords[i], coords[j])
print("\nMatriks Jarak:")
print(dist_matrix)

Koordinat kota:
Kota 0: (20, 23)
Kota 1: (23, 3)
Kota 2: (3, 12)
Kota 3: (12, 6)
Kota 4: (6, 20)

Matriks Jarak:
[[ 0.   20.22 20.25 18.79 14.32]
 [20.22  0.   21.93 11.4  24.04]
 [20.25 21.93  0.   10.82  8.54]
 [18.79 11.4  10.82  0.   15.23]
 [14.32 24.04  8.54 15.23  0.  ]]


In [5]:
# --- Fungsi Gambar Jalur TSP ---
def plot_path(coords, path, title):
    x = [coords[i][0] for i in path]
    y = [coords[i][1] for i in path]
    plt.figure(figsize=(6, 4))
    plt.plot(x, y, 'bo-', linewidth=2, markersize=6)
    for i, (x_, y_) in enumerate(coords):
        plt.text(x_ + 0.2, y_ + 0.2, f"{i}")
    plt.title(title)
    plt.grid(True)
    plt.show()

# --- Scatter Plot: hanya tampilkan jika tersedia ---
if 'path_bb' in locals():
    plot_path(coords, path_bb, "Rute Branch and Bound")

if 'path_greedy' in locals():
    plot_path(coords, path_greedy, "Rute Greedy")

# --- Bar Plot: hanya jika keduanya tersedia ---
if 'cost_bb' in locals() and 'cost_greedy' in locals():
    df = pd.DataFrame({
        "Algoritma": ["Branch and Bound", "Greedy"],
        "Jarak": [cost_bb, cost_greedy]
    })
    sns.set(style="whitegrid")
    plt.figure(figsize=(6,4))
    sns.barplot(x="Algoritma", y="Jarak", data=df, palette="viridis")
    plt.title("Perbandingan Total Jarak TSP")
    plt.ylabel("Total Distance")
    plt.show()


In [ ]:
# --- Branch and Bound ---
class Node:
    def __init__(self, path, cost, level, bound):
        self.path = path
        self.cost = cost
        self.level = level
        self.bound = bound

    def __lt__(self, other):
        return self.bound < other.bound

def bound(node, dist_matrix):
    cost = node.cost
    n = len(dist_matrix)
    visited = set(node.path)
    for i in range(n):
        if i not in visited:
            min_edge = min([dist_matrix[i][j] for j in range(n) if i != j and j not in visited], default=0)
            cost += min_edge
    return cost

def tsp_branch_and_bound(dist_matrix):
    n = len(dist_matrix)
    root = Node(path=[0], cost=0, level=0, bound=0)
    root.bound = bound(root, dist_matrix)
    
    pq = []
    heapq.heappush(pq, root)
    min_cost = float('inf')
    best_path = []

    while pq:
        curr = heapq.heappop(pq)
        if curr.bound >= min_cost:
            continue
        if curr.level == n - 1:
            last_to_first = dist_matrix[curr.path[-1]][0]
            total_cost = curr.cost + last_to_first
            if total_cost < min_cost:
                min_cost = total_cost
                best_path = curr.path + [0]
        else:
            for i in range(n):
                if i not in curr.path:
                    new_path = curr.path + [i]
                    new_cost = curr.cost + dist_matrix[curr.path[-1]][i]
                    child = Node(new_path, new_cost, curr.level + 1, 0)
                    child.bound = bound(child, dist_matrix)
                    if child.bound < min_cost:
                        heapq.heappush(pq, child)

    return best_path, min_cost

if algoritma in ["bb", "semua"]:
    path_bb, cost_bb = tsp_branch_and_bound(dist_matrix)
    print("\n[Branch and Bound]")
    print("Path:", path_bb)
    print("Cost:", cost_bb)

In [7]:
# --- Greedy ---
def tsp_greedy(dist_matrix):
    n = len(dist_matrix)
    visited = [False] * n
    path = [0]
    total_cost = 0
    visited[0] = True

    for _ in range(n-1):
        last = path[-1]
        next_city = min((dist_matrix[last][j], j) for j in range(n) if not visited[j])[1]
        path.append(next_city)
        total_cost += dist_matrix[last][next_city]
        visited[next_city] = True

    total_cost += dist_matrix[path[-1]][0]
    path.append(0)
    return path, total_cost

if algoritma in ["greedy", "semua"]:
    path_greedy, cost_greedy = tsp_greedy(dist_matrix)
    print("\n[Greedy]")
    print("Path:", path_greedy)
    print("Cost:", cost_greedy)

In [11]:
# --- Fungsi Gambar Jalur TSP ---
def plot_path(coords, path, title):
    x = [coords[i][0] for i in path]
    y = [coords[i][1] for i in path]
    plt.figure(figsize=(6, 4))
    plt.plot(x, y, 'bo-', linewidth=2, markersize=6)
    for i, (x_, y_) in enumerate(coords):
        plt.text(x_ + 0.2, y_ + 0.2, f"{i}")
    plt.title(title)
    plt.grid(True)
    plt.show()

# --- Scatter Plot: hanya tampilkan jika variabelnya sudah dibuat ---
if 'path_bb' in locals():
    plot_path(coords, path_bb, "Rute Branch and Bound")

if 'path_greedy' in locals():
    plot_path(coords, path_greedy, "Rute Greedy")

# --- Bar Plot Seaborn: hanya jika keduanya tersedia ---
if 'cost_bb' in locals() and 'cost_greedy' in locals():
    df = pd.DataFrame({
        "Algoritma": ["Branch and Bound", "Greedy"],
        "Jarak": [cost_bb, cost_greedy]
    })
    sns.set(style="whitegrid")
    plt.figure(figsize=(6,4))
    sns.barplot(x="Algoritma", y="Jarak", data=df, palette="viridis")
    plt.title("Perbandingan Total Jarak TSP")
    plt.ylabel("Total Distance")
    plt.show()
